# Khái niệm cơ bản

Hàm mật độ xác suất:
+ biểu thị mức độ tập trung của xác suất quanh điểm đó, giá trị lớn thì khả năng xuất hiện của biến ngẫu nhiên tại điểm đó lớn hơn.

Hàm phân phối xác suất:
+ là xác suất mà biến ngẫu nhiên có giá trị nhỏ hơn hoặc bằng một giá trị nào đó.
+ là tích phân của hàm mật độ xác suất với khoảng từ - vô cực đến x.

# Guassian Naive Bayes

In [19]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    df[iris.feature_names], df['target'], test_size=0.2, random_state=42, stratify=df['target']
)

In [22]:
import numpy as np
np.mean(X_train, axis=0)

sepal length (cm)    5.841667
sepal width (cm)     3.048333
petal length (cm)    3.770000
petal width (cm)     1.205000
dtype: float64

In [47]:
class GaussianNaiveBayes:
    def __init__(self):
        self.means = None
        self.variances = None
        self.labels = None
        self.nclasses = None
        self.mean = {}
        self.variance = {}
        self.pk = {}
        self.pxk = []

    def get_mean_variance(self, X, label):
        if self.mean.get(label) is None:
            self.mean[label] = np.mean(X, axis=0)
        if self.variance.get(label) is None:
            self.variance[label] = np.var(X, axis=0)
        # self.mean = np.mean(X, axis=0)
        # self.variance = np.var(X, axis=0)
        # return self.mean, self.variance
    
    def adf(self, X, mean, variance):
        return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-((X - mean) ** 2) / (2 * variance))
    
    def fit(self, X, y):
        self.nclasses = len(np.unique(y))
        self.labels = np.unique(y)
        for i in np.unique(y):
            X_i = X[y == i]
            self.get_mean_variance(X_i, i)
            self.pk[i] = len(X_i) / len(X)

            # self.pxk.append(self.adf(X_i, self.mean, self.variance))

    def predict(self, X_test):
        res = []
        for i in X_test:
            value = []
            for j in self.labels:
                pxk = self.adf(i, self.mean[j], self.variance[j])
                value.append(np.prod(pxk) * self.pk[j])
            res.append(self.labels[np.argmax(value)])
        return res
    
    def evaluate(self, y_test, y_pred):
        return np.sum(y_test == y_pred) / len(y_test) * 100


In [57]:
gnb = GaussianNaiveBayes()
gnb.fit(X_train.values, y_train.values)
predict = gnb.predict(X_test.values)
print(predict)
print('Độ chính xác:', gnb.evaluate(y_test.values, predict))

[np.int64(0), np.int64(2), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(2), np.int64(1), np.int64(2), np.int64(2), np.int64(2), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(2), np.int64(0), np.int64(2), np.int64(1), np.int64(2), np.int64(2), np.int64(2), np.int64(1), np.int64(0), np.int64(2), np.int64(0)]
Độ chính xác: 96.66666666666667


# Multinomial Naive Bayes

In [39]:
from sklearn.datasets import fetch_20newsgroups
from scipy.sparse import coo_matrix

# data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# print(data.data[0])  # Văn bản đầu tiên
# print(data.target[0])  # Nhãn của văn bản đầu tiên

In [51]:
class MultinomialNaiveBayes:
    def __init__(self):
        self.pk = {}
        self.pxk = {}
        self.labels = None
        self.nclasses = None

    def fit(self, X, y):
        self.nclasses = len(np.unique(y))
        self.labels = np.unique(y)
        
        for i in np.unique(y):
            if self.pxk.get(i) is None:
                self.pxk[i] = {}
            
            for j in range(X.shape[1]):
                if self.pxk[i].get(j) is None:
                    self.pxk[i][j] = {}
        

        for i in np.unique(y):
            X_i = X[y == i]
            self.pk[i] = len(X_i) / len(X)
            for j in range(X.shape[1]):
                for k in np.unique(X[:, j]):
                    self.pxk[i][j][k] = (np.sum(X_i[:, j] == k) + 1) / (len(X_i) + len(np.unique(X[:, j])))

    def predict(self, X_test):
        res = []
        for i in X_test:
            value = []
            for j in self.labels:
                pxk = []
                for index, k in enumerate(i):
                    pxk.append(self.pxk[j][index].get(k, 1e-6))
                value.append(np.prod(pxk) * self.pk[j])
            res.append(self.labels[np.argmax(value)])
        return res

In [36]:
train_data_fn = 'data/train-features.txt'
test_data_fn = 'data/test-features.txt'
train_label_fn = 'data/train-labels.txt'
test_label_fn = 'data/test-labels.txt'

In [40]:
nwords = 2500

def read_data(data_fn, label_fn):
  ## read label_fn
  with open(label_fn) as f:
    content = f.readlines()
  label = [int(x.strip()) for x in content]
  with open(data_fn) as f:
    content = f.readlines()
  # remove '\n' at the end of each line
  content = [x.strip() for x in content]

  dat = np.zeros((len(content), 3), dtype = int)

  for i, line in enumerate(content):
    a = line.split(' ')
    dat[i, :] = np.array([int(a[0]), int(a[1]), int(a[2])])

  data = coo_matrix((dat[:, 2], (dat[:, 0] - 1, dat[:, 1] - 1)),\
    shape=(len(label), nwords))
  return (data, label)


In [41]:
(train_data, train_label) = read_data(train_data_fn, train_label_fn)
(test_data, test_label) = read_data(test_data_fn, test_label_fn)

In [43]:
train_data = train_data.toarray()
test_data = test_data.toarray()
print(train_data.shape)

(700, 2500)


In [45]:
train_data

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 1,  0,  0, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  0],
       [ 3,  2, 17, ...,  0,  0,  3]], shape=(700, 2500))

In [ ]:
mnb = MultinomialNaiveBayes()
mnb.fit(train_data, train_label)
predict = mnb.predict(test_data)
# predict = np.array(predict)
# test_label = np.array(test_label)
# print('Độ chính xác:', np.sum(predict == test_label) / len(test_label) * 100)
print('Độ chính xác:', np.sum(predict == test_label) / len(test_label) * 100)


Độ chính xác: 74.61538461538461
